In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error as mse
from scipy.stats import entropy
import warnings

from causalml.inference.meta import LRSRegressor
from causalml.inference.meta import XGBTRegressor, MLPTRegressor
from causalml.inference.meta import BaseXRegressor, BaseRRegressor, BaseSRegressor, BaseTRegressor
from causalml.inference.tf import DragonNet
from causalml.match import NearestNeighborMatch, MatchOptimizer, create_table_one
from causalml.propensity import ElasticNetPropensityModel
from causalml.metrics import *


import torch
import logging
from causalml.inference.nn import CEVAE

import os, sys

%matplotlib inline

warnings.filterwarnings('ignore')
plt.style.use('fivethirtyeight')
sns.set_palette('Paired')
plt.rcParams['figure.figsize'] = (12,8)

2024-04-04 14:08:24.514584: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-04 14:08:25.778290: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-04 14:08:25.778319: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-04 14:08:25.780277: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-04 14:08:26.000825: I tensorflow/core/platform/cpu_feature_guar

## BPIC 2017 

In [2]:
df = pd.read_csv("bpi2017_final.csv")
print(df.columns)
df.head()

Index(['case:concept:name', 'NumberOfOffers', 'Action', 'org:resource',
       'concept:name', 'EventOrigin', 'lifecycle:transition', 'time:timestamp',
       'case:LoanGoal', 'case:ApplicationType', 'case:RequestedAmount',
       'FirstWithdrawalAmount', 'NumberOfTerms', 'Accepted', 'MonthlyCost',
       'Selected', 'CreditScore', 'OfferedAmount', 'treatedCase',
       'caseSuccesful', 'treatmentSuccess', 'offerNumber', 'offerSuccess',
       'treatmentOffer', 'timeApplication', 'weekdayApplication'],
      dtype='object')


,case:concept:name,NumberOfOffers,Action,org:resource,concept:name,EventOrigin,lifecycle:transition,time:timestamp,case:LoanGoal,case:ApplicationType,...,CreditScore,OfferedAmount,treatedCase,caseSuccesful,treatmentSuccess,offerNumber,offerSuccess,treatmentOffer,timeApplication,weekdayApplication
0,0.0,1.0,0.0,0.0,4.0,0.0,1.0,651433.0,10.0,1.0,...,0.0,5000.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000,2.0
1,0.0,1.0,4.0,0.0,8.0,0.0,1.0,651434.0,10.0,1.0,...,0.0,5000.0,0.0,0.0,0.0,1.0,0.0,0.0,0.061,2.0
2,0.0,1.0,0.0,0.0,22.0,2.0,3.0,651435.0,10.0,1.0,...,0.0,5000.0,0.0,0.0,0.0,1.0,0.0,0.0,0.290,2.0
3,0.0,1.0,1.0,0.0,22.0,2.0,6.0,651437.0,10.0,1.0,...,0.0,5000.0,0.0,0.0,0.0,1.0,0.0,0.0,66.613,2.0
4,0.0,1.0,0.0,0.0,21.0,2.0,3.0,651438.0,10.0,1.0,...,0.0,5000.0,0.0,0.0,0.0,1.0,0.0,0.0,66.620,2.0


In [3]:
feature_names = ['NumberOfOffers', 'Action', 'org:resource',
       'concept:name', 'EventOrigin', 'lifecycle:transition', 'time:timestamp',
       'case:LoanGoal', 'case:ApplicationType', 'case:RequestedAmount',
       'FirstWithdrawalAmount', 'NumberOfTerms', 'Accepted', 'MonthlyCost',
       'CreditScore', 'OfferedAmount', 'offerNumber','timeApplication', 'weekdayApplication']

In [4]:
y_outcome = df['offerSuccess'].values
x_feature = df[feature_names].values
t_treatment = np.array([np.array([value]) for value in df['treatmentOffer']])

In [ ]:
dragon = DragonNet(neurons_per_layer=200, targeted_reg=True)
dragon_ite = dragon.fit_predict(x_feature, t_treatment, y_outcome, return_components=False)
dragon_ate = dragon_ite.mean()

In [ ]:
print(dragon_ite)
print(dragon_ate)

In [6]:
y = 1

# Calculate statistics
data = np.reshape(dragon_ite, -1)
minimum = np.min(data)
first_quartile = np.percentile(data, 25)
median = np.median(data)
third_quartile = np.percentile(data, 75)
maximum = np.max(data)

# Interquartile range (IQR)
iqr = third_quartile - first_quartile

# Define upper and lower bounds for outliers
upper_bound = third_quartile + 1.5 * iqr
lower_bound = first_quartile - 1.5 * iqr

# Detect outliers
outliers = data[(data < lower_bound) | (data > upper_bound)]

# Print the statistics
print("Minimum:", minimum)
print("First Quartile:", first_quartile)
print("Median:", median)
print("Third Quartile:", third_quartile)
print("Maximum:", maximum)
print("Interquartile Range:", iqr)
print("Upper Bound (Outliers):", upper_bound)
print("Lower Bound (Outliers):", lower_bound)
print("Outliers:", outliers)

ite_dragon = [minimum, first_quartile, median, third_quartile, maximum, iqr, upper_bound, lower_bound]

Minimum: 0.12840658
First Quartile: 0.1284065842628479
Median: 0.12840658
Third Quartile: 0.1284065842628479
Maximum: 0.12840658
Interquartile Range: 0.0
Upper Bound (Outliers): 0.1284065842628479
Lower Bound (Outliers): 0.1284065842628479
Outliers: []


### CEVAE Model

This module implements the Causal Effect Variational Autoencoder [1]

[1] C. Louizos, U. Shalit, J. Mooij, D. Sontag, R. Zemel, M. Welling (2017).
Causal Effect Inference with Deep Latent-Variable Models.
http://papers.nips.cc/paper/7223-causal-effect-inference-with-deep-latent-variable-models.pdf
https://github.com/AMLab-Amsterdam/CEVAE

In [4]:
# Split data to training and testing samples for model validation (next section)
df_train, df_test = train_test_split(df, test_size=0.2, random_state=11101)

In [5]:
X_train = df_train[feature_names].values
X_test = df_test[feature_names].values
treatment_train = df_train['treatmentOffer'].values
treatment_test = df_test['treatmentOffer'].values
y_train = df_train['offerSuccess'].values
y_test = df_test['offerSuccess'].values

In [6]:
# cevae model settings
outcome_dist = "normal"
latent_dim = 20
hidden_dim = 200
num_epochs = 5
batch_size = 1000
learning_rate = 0.001
learning_rate_decay = 0.01
num_layers = 2

In [7]:
cevae = CEVAE(outcome_dist=outcome_dist,
              latent_dim=latent_dim,
              hidden_dim=hidden_dim,
              num_epochs=num_epochs,
              batch_size=batch_size,
              learning_rate=learning_rate,
              learning_rate_decay=learning_rate_decay,
              num_layers=num_layers)

In [8]:
# fit
losses = cevae.fit(X=torch.tensor(X_train, dtype=torch.float),
                   treatment=torch.tensor(treatment_train, dtype=torch.float),
                   y=torch.tensor(y_train, dtype=torch.float))

INFO 	 Training with 959 minibatches per epoch
DEBUG 	 step     0 loss = 39.432
DEBUG 	 step   100 loss = 21.5587
DEBUG 	 step   200 loss = 17.7834
DEBUG 	 step   300 loss = 16.1056
DEBUG 	 step   400 loss = 14.7949
DEBUG 	 step   500 loss = 13.9928
DEBUG 	 step   600 loss = 12.7464
DEBUG 	 step   700 loss = 11.3728
DEBUG 	 step   800 loss = 11.5583
DEBUG 	 step   900 loss = 11.1287
DEBUG 	 step  1000 loss = 10.064
DEBUG 	 step  1100 loss = 10.4556
DEBUG 	 step  1200 loss = 9.80264
DEBUG 	 step  1300 loss = 9.04074
DEBUG 	 step  1400 loss = 8.62775
DEBUG 	 step  1500 loss = 8.42838
DEBUG 	 step  1600 loss = 7.67158
DEBUG 	 step  1700 loss = 8.49774
DEBUG 	 step  1800 loss = 8.13176
DEBUG 	 step  1900 loss = 7.54507
DEBUG 	 step  2000 loss = 7.20558
DEBUG 	 step  2100 loss = 7.25838
DEBUG 	 step  2200 loss = 7.25943
DEBUG 	 step  2300 loss = 7.16487
DEBUG 	 step  2400 loss = 6.76904
DEBUG 	 step  2500 loss = 7.35741
DEBUG 	 step  2600 loss = 7.30728
DEBUG 	 step  2700 loss = 6.72472
DEB

In [9]:
# predict
ate_val = cevae.predict(X_test)

INFO 	 Evaluating 240 minibatches
DEBUG 	 batch ate = -0.0518396
DEBUG 	 batch ate = -0.0578103
DEBUG 	 batch ate = 0.0179996
DEBUG 	 batch ate = -0.0672696
DEBUG 	 batch ate = -0.0212976
DEBUG 	 batch ate = -0.0394065
DEBUG 	 batch ate = -0.071669
DEBUG 	 batch ate = -0.0345337
DEBUG 	 batch ate = -0.0644459
DEBUG 	 batch ate = -0.0492067
DEBUG 	 batch ate = -0.0511732
DEBUG 	 batch ate = -0.0738739
DEBUG 	 batch ate = -0.0653151
DEBUG 	 batch ate = -0.0608201
DEBUG 	 batch ate = -0.0718545
DEBUG 	 batch ate = -0.0776044
DEBUG 	 batch ate = -0.0560759
DEBUG 	 batch ate = -0.089948
DEBUG 	 batch ate = -0.0721967
DEBUG 	 batch ate = -0.0310648
DEBUG 	 batch ate = -0.0875605
DEBUG 	 batch ate = -0.056983
DEBUG 	 batch ate = -0.0885981
DEBUG 	 batch ate = -0.0835431
DEBUG 	 batch ate = -0.0559851
DEBUG 	 batch ate = -0.096993
DEBUG 	 batch ate = -0.04468
DEBUG 	 batch ate = -0.0763461
DEBUG 	 batch ate = -0.0586184
DEBUG 	 batch ate = -0.0412979
DEBUG 	 batch ate = -0.0367664
DEBUG 	 batc

### Input results in result df

In [ ]:
#for Dragonnet
%store -r df_results
lib = "CausalML"
method = "Dragonnet"

if method in df_results['method'].values:
    df_results.loc[df_results['method'] == method, 'ATE'] = dragon_ate
    df_results.loc[df_results['method'] == method, 'ITE'] = ite_dragon

else:
    df_results = df_results._append({'method': method, 'ATE': dragon_ate, 'ITE': ite_dragon, 'Library': lib}, ignore_index=True)

%store df_results

In [10]:
# for CEVAE
%store -r df_results
ate = ate_val.mean()
lib = "CausalML"
method = "CEVAE"

if method in df_results['method'].values:
    df_results.loc[df_results['method'] == method, 'ATE'] = ate

else:
    df_results = df_results._append({'method': method, 'ATE': ate, 'Library': lib}, ignore_index=True)

%store df_results

Stored 'df_results' (DataFrame)


## Synthetic Data

In [2]:
df_synth = pd.read_csv("synthetic_dataset.csv")
df_synth.head()
synthetic_features = ['NumberOfOffers', 'concept:name',
       'lifecycle:transition', 'time:timestamp', 'elementId', 'resourceId',
       'weekdayApplication', 'timeApplication']

t_treatment=np.array([np.array([value]) for value in df_synth['treatment']])
x_feature = df_synth[synthetic_features].values
y_outcome=df_synth['treatmentSuccess'].values

In [3]:
synth_dragon = DragonNet(neurons_per_layer=100, targeted_reg=True)
synth_dragon_ite = synth_dragon.fit_predict(x_feature, t_treatment, y_outcome, return_components=False)
synth_dragon_ate = synth_dragon_ite.mean()
print(synth_dragon_ite, synth_dragon_ate)

Epoch 1/30
2795/2795 [==============================] - 35s 10ms/step - loss: 1363.5438 - regression_loss: 657.3325 - binary_classification_loss: 44.0058 - treatment_accuracy: 0.6488 - track_epsilon: 0.0119 - val_loss: 65.0797 - val_regression_loss: 8.0731 - val_binary_classification_loss: 49.5177 - val_treatment_accuracy: 0.5405 - val_track_epsilon: 0.0069 - lr: 0.0010
Epoch 2/30
2795/2795 [==============================] - 24s 8ms/step - loss: 2042.7755 - regression_loss: 1000.8886 - binary_classification_loss: 38.6559 - treatment_accuracy: 0.7872 - track_epsilon: 0.0030 - val_loss: 16.7200 - val_regression_loss: 1.2573 - val_binary_classification_loss: 12.0500 - val_treatment_accuracy: 0.9596 - val_track_epsilon: 0.0031 - lr: 0.0010
Epoch 3/30
2795/2795 [==============================] - 22s 8ms/step - loss: 17.7302 - regression_loss: 1.2549 - binary_classification_loss: 12.9890 - treatment_accuracy: 0.9569 - track_epsilon: 0.0018 - val_loss: 11.4919 - val_regression_loss: 0.2895 - 

In [26]:
y = 1

# Calculate statistics
data = np.reshape(synth_dragon_ite, -1)
minimum = np.min(data)
first_quartile = np.percentile(data, 25)
median = np.median(data)
third_quartile = np.percentile(data, 75)
maximum = np.max(data)

# Interquartile range (IQR)
iqr = third_quartile - first_quartile

# Define upper and lower bounds for outliers
upper_bound = third_quartile + 1.5 * iqr
lower_bound = first_quartile - 1.5 * iqr

# Detect outliers
outliers = data[(data < lower_bound) | (data > upper_bound)]

# Print the statistics
print("Minimum:", minimum)
print("First Quartile:", first_quartile)
print("Median:", median)
print("Third Quartile:", third_quartile)
print("Maximum:", maximum)
print("Interquartile Range:", iqr)
print("Upper Bound (Outliers):", upper_bound)
print("Lower Bound (Outliers):", lower_bound)
print("Outliers:", outliers)

ite_synth_dragon = [minimum, first_quartile, median, third_quartile, maximum, iqr, upper_bound, lower_bound]

Minimum: nan
First Quartile: nan
Median: nan
Third Quartile: nan
Maximum: nan
Interquartile Range: nan
Upper Bound (Outliers): nan
Lower Bound (Outliers): nan
Outliers: []


In [3]:
# Split data to training and testing samples for model validation (next section)
df_synth_train, df_synth_test = train_test_split(df_synth, test_size=0.2, random_state=11101)

In [4]:
X_train = df_synth_train[synthetic_features].values
X_test = df_synth_test[synthetic_features].values
treatment_train = df_synth_train['treatment'].values
treatment_test = df_synth_test['treatment'].values
y_train = df_synth_train['treatmentSuccess'].values
y_test = df_synth_test['treatmentSuccess'].values

In [5]:
# cevae model settings
outcome_dist = "normal"
latent_dim = 20
hidden_dim = 200
num_epochs = 5
batch_size = 1000
learning_rate = 0.001
learning_rate_decay = 0.01
num_layers = 2

cevae = CEVAE(outcome_dist=outcome_dist,
              latent_dim=latent_dim,
              hidden_dim=hidden_dim,
              num_epochs=num_epochs,
              batch_size=batch_size,
              learning_rate=learning_rate,
              learning_rate_decay=learning_rate_decay,
              num_layers=num_layers)

In [6]:
# fit
synth_losses = cevae.fit(X=torch.tensor(X_train, dtype=torch.float),
                   treatment=torch.tensor(treatment_train, dtype=torch.float),
                   y=torch.tensor(y_train, dtype=torch.float))

INFO 	 Training with 179 minibatches per epoch
DEBUG 	 step     0 loss = 22.5471
DEBUG 	 step   100 loss = 5.76006
DEBUG 	 step   200 loss = 3.36875
DEBUG 	 step   300 loss = 2.09083
DEBUG 	 step   400 loss = 1.69662
DEBUG 	 step   500 loss = 0.765509
DEBUG 	 step   600 loss = 0.490193
DEBUG 	 step   700 loss = 0.127225
DEBUG 	 step   800 loss = 0.150899


In [7]:
# predict
synth_ite = cevae.predict(X_test)

INFO 	 Evaluating 45 minibatches
DEBUG 	 batch ate = 1.96903
DEBUG 	 batch ate = 1.96812
DEBUG 	 batch ate = 1.96889
DEBUG 	 batch ate = 1.96825
DEBUG 	 batch ate = 1.96991
DEBUG 	 batch ate = 1.96704
DEBUG 	 batch ate = 1.96895
DEBUG 	 batch ate = 1.96887
DEBUG 	 batch ate = 1.96924
DEBUG 	 batch ate = 1.96949
DEBUG 	 batch ate = 1.96844
DEBUG 	 batch ate = 1.96864
DEBUG 	 batch ate = 1.96885
DEBUG 	 batch ate = 1.96882
DEBUG 	 batch ate = 1.96859
DEBUG 	 batch ate = 1.96905
DEBUG 	 batch ate = 1.9686
DEBUG 	 batch ate = 1.97003
DEBUG 	 batch ate = 1.96898
DEBUG 	 batch ate = 1.96913
DEBUG 	 batch ate = 1.96972
DEBUG 	 batch ate = 1.96717
DEBUG 	 batch ate = 1.96924
DEBUG 	 batch ate = 1.96976
DEBUG 	 batch ate = 1.96751
DEBUG 	 batch ate = 1.96974
DEBUG 	 batch ate = 1.96928
DEBUG 	 batch ate = 1.96844
DEBUG 	 batch ate = 1.96896
DEBUG 	 batch ate = 1.96951
DEBUG 	 batch ate = 1.96915
DEBUG 	 batch ate = 1.96845
DEBUG 	 batch ate = 1.96846
DEBUG 	 batch ate = 1.96725
DEBUG 	 batch at

In [8]:
synth_ate = synth_ite.mean()

In [9]:
import evaluation_metrics
true_ate = 2
boxplot = evaluation_metrics.boxplot_ite(synth_ite)
metrics = evaluation_metrics.evaluation_metrics(true_ate, synth_ite)
print(boxplot, metrics)

[1.8834785, 1.950274258852005, 1.9614615, 1.9918553829193115, 2.3098311, 0.04158112406730652, 2.0542270690202713, 1.8879025727510452] [0.0014018206, 0.031313766, 0.037440896, 0.031222465, 0.018720448]


In [10]:
%store -r df_synthetic_results_metric

method = "CEVAE"
ate = synth_ate
ite = boxplot
metric = metrics

df_synthetic_results_metric = df_synthetic_results_metric._append({'method': method, 'ATE': ate, 'ITE': ite, 'metrics': metric}, ignore_index=True)

print(df_synthetic_results_metric)
%store df_synthetic_results_metric

                                           method  \
0                                    S-Learner LR   
1                                   XGBTRegressor   
2                              BaseTRegressor XGB   
3                               BaseTRegressor LR   
4                              BaseXRegressor XGB   
5                               BaseXRegressor LR   
6   BaseXRegressor XGB (without propensity score)   
7    BaseXRegressor LR (without propensity score)   
8                              BaseRRegressor XGB   
9                               BaseRRegressor LR   
10        BaseRRegressor XGB (with random weight)   
11  BaseRRegressor XGB (without propensity score)   
12                           Neural Network (MLP)   
13                                         BCAUSS   
14                                          CEVAE   

                                                  ITE       ATE  \
0   [1.999999999999998, 1.9999999999999987, 1.9999...  2.000000   
1            [2.0

## Refutation Tests

In [2]:
import os
# Define folder path
folder_path = "./evaluationDatasets/Placebo/"

# List to store treatment effects
ate_values = []

feature_names = ['NumberOfOffers', 'Action', 'org:resource',
       'concept:name', 'EventOrigin', 'lifecycle:transition', 'time:timestamp',
       'case:LoanGoal', 'case:ApplicationType', 'case:RequestedAmount',
       'FirstWithdrawalAmount', 'NumberOfTerms', 'Accepted', 'MonthlyCost',
       'CreditScore', 'OfferedAmount', 'offerNumber','timeApplication', 'weekdayApplication']

columns_to_drop = ['offerSuccess', 'treatmentOffer']

# Iterate through files in the folder
for file_name in os.listdir(folder_path):
    print(file_name)
    # Read CSV file
    file_path = os.path.join(folder_path, file_name)
    refutation = pd.read_csv(file_path)
    
    y_outcome = refutation['offerSuccess'].values
    x_feature = refutation[feature_names].values
    #x_feature = refutation.drop(columns=columns_to_drop).values
    t_treatment = np.array([np.array([value]) for value in refutation['treatmentOffer']])
    
    dragon = DragonNet(neurons_per_layer=200, targeted_reg=True)
    dragon_ite = dragon.fit_predict(x_feature, t_treatment, y_outcome, return_components=False)
    dragon_ate = dragon_ite.mean()
    print(dragon_ate)
    ate_values.append(dragon_ate)

print("Average ATE: ")
print(sum(ate_values) / len(ate_values))

placeboDataset2.csv
Epoch 1/30
14979/14979 [==============================] - 40s 3ms/step - loss: 2940743.5000 - regression_loss: 1470298.7500 - binary_classification_loss: 222.7343 - treatment_accuracy: 0.4964 - track_epsilon: 0.0036 - val_loss: 274.2520 - val_regression_loss: 23.6782 - val_binary_classification_loss: 222.9408 - val_treatment_accuracy: 0.4959 - val_track_epsilon: 6.2120e-05 - lr: 0.0010
Epoch 2/30
14979/14979 [==============================] - 38s 3ms/step - loss: 35492.7812 - regression_loss: 17642.8223 - binary_classification_loss: 194.8339 - treatment_accuracy: 0.4971 - track_epsilon: 0.0013 - val_loss: 79.3366 - val_regression_loss: 15.3410 - val_binary_classification_loss: 44.6888 - val_treatment_accuracy: 0.4959 - val_track_epsilon: 0.0022 - lr: 0.0010
Epoch 3/30
14979/14979 [==============================] - 36s 2ms/step - loss: 567.1451 - regression_loss: 259.7182 - binary_classification_loss: 44.7305 - treatment_accuracy: 0.5002 - track_epsilon: 0.0104 - val

In [ ]:
import os
# Define folder path
folder_path = "./evaluationDatasets/Replace/"

# List to store treatment effects
ate_values = []

feature_names = ['NumberOfOffers', 'Action', 'org:resource',
       'concept:name', 'EventOrigin', 'lifecycle:transition', 'time:timestamp',
       'case:LoanGoal', 'case:ApplicationType', 'case:RequestedAmount',
       'FirstWithdrawalAmount', 'NumberOfTerms', 'Accepted', 'MonthlyCost',
       'CreditScore', 'OfferedAmount', 'offerNumber','timeApplication', 'weekdayApplication']

columns_to_drop = ['offerSuccess', 'treatmentOffer']

# cevae model settings
outcome_dist = "normal"
latent_dim = 20
hidden_dim = 200
num_epochs = 5
batch_size = 1000
learning_rate = 0.001
learning_rate_decay = 0.01
num_layers = 2

cevae = CEVAE(outcome_dist=outcome_dist,
              latent_dim=latent_dim,
              hidden_dim=hidden_dim,
              num_epochs=num_epochs,
              batch_size=batch_size,
              learning_rate=learning_rate,
              learning_rate_decay=learning_rate_decay,
              num_layers=num_layers)

# Iterate through files in the folder
for file_name in os.listdir(folder_path):
    print(file_name)
    # Read CSV file
    file_path = os.path.join(folder_path, file_name)
    refutation = pd.read_csv(file_path)
    
    # Split data to training and testing samples for model validation
    train, test = train_test_split(refutation, test_size=0.2, random_state=11101)
    
    X_train = train[feature_names].values
    X_test = test[feature_names].values
    # X_train = train.drop(columns=columns_to_drop).values
    # X_test = test.drop(columns=columns_to_drop).values
    treatment_train = train['treatmentOffer'].values
    y_train = train['offerSuccess'].values

    # fit
    losses = cevae.fit(X=torch.tensor(X_train, dtype=torch.float),
                       treatment=torch.tensor(treatment_train, dtype=torch.float),
                       y=torch.tensor(y_train, dtype=torch.float))
    
    # predict
    ate_val = cevae.predict(X_test)
    print(ate_val.mean())
    ate_values.append(ate_val.mean())

print("Average ATE: ")
print(sum(ate_values) / len(ate_values))

randomReplacedDataset2.csv


INFO 	 Training with 959 minibatches per epoch
DEBUG 	 step     0 loss = 37.0999
DEBUG 	 step   100 loss = 17.5016
DEBUG 	 step   200 loss = 15.2348
DEBUG 	 step   300 loss = 14.2043
DEBUG 	 step   400 loss = 12.7082
DEBUG 	 step   500 loss = 11.5089
DEBUG 	 step   600 loss = 10.0316
DEBUG 	 step   700 loss = 9.32373
DEBUG 	 step   800 loss = 8.90895
DEBUG 	 step   900 loss = 8.07123
DEBUG 	 step  1000 loss = 7.07197
DEBUG 	 step  1100 loss = 6.50148
DEBUG 	 step  1200 loss = 6.31736
DEBUG 	 step  1300 loss = 5.42301
DEBUG 	 step  1400 loss = 4.61341
DEBUG 	 step  1500 loss = 5.6245
DEBUG 	 step  1600 loss = 5.0767
DEBUG 	 step  1700 loss = 4.01805
DEBUG 	 step  1800 loss = 4.58147
DEBUG 	 step  1900 loss = 3.4589
DEBUG 	 step  2000 loss = 3.49854
DEBUG 	 step  2100 loss = 3.1304
DEBUG 	 step  2200 loss = 2.93198
DEBUG 	 step  2300 loss = 2.48879
DEBUG 	 step  2400 loss = 2.56436
DEBUG 	 step  2500 loss = 2.60999
DEBUG 	 step  2600 loss = 2.55109
DEBUG 	 step  2700 loss = 2.06989
DEBUG

-0.15435798
randomReplacedDataset3.csv


INFO 	 Training with 959 minibatches per epoch
DEBUG 	 step     0 loss = 36.2184
DEBUG 	 step   100 loss = 36.3472
DEBUG 	 step   200 loss = 36.7701
DEBUG 	 step   300 loss = 36.2554
DEBUG 	 step   400 loss = 37.8101
DEBUG 	 step   500 loss = 36.4088
DEBUG 	 step   600 loss = 36.1793
DEBUG 	 step   700 loss = 36.4583
DEBUG 	 step   800 loss = 36.369
DEBUG 	 step   900 loss = 38.4637
DEBUG 	 step  1000 loss = 37.1505
DEBUG 	 step  1100 loss = 36.6821
DEBUG 	 step  1200 loss = 36.9666
DEBUG 	 step  1300 loss = 36.7821
DEBUG 	 step  1400 loss = 37.3813
DEBUG 	 step  1500 loss = 36.5468
DEBUG 	 step  1600 loss = 37.4426
DEBUG 	 step  1700 loss = 37.0136
DEBUG 	 step  1800 loss = 36.7318
DEBUG 	 step  1900 loss = 36.6915
DEBUG 	 step  2000 loss = 36.3755
DEBUG 	 step  2100 loss = 36.8656
DEBUG 	 step  2200 loss = 36.4642
DEBUG 	 step  2300 loss = 36.8496
DEBUG 	 step  2400 loss = 37.5386
DEBUG 	 step  2500 loss = 36.354
DEBUG 	 step  2600 loss = 36.5195
DEBUG 	 step  2700 loss = 37.0572
DEB

0.049451213
randomReplacedDataset7.csv


INFO 	 Training with 959 minibatches per epoch
DEBUG 	 step     0 loss = 38.3737
DEBUG 	 step   100 loss = 37.21
DEBUG 	 step   200 loss = 38.4292
DEBUG 	 step   300 loss = 36.9215
DEBUG 	 step   400 loss = 36.8218
DEBUG 	 step   500 loss = 36.6969
DEBUG 	 step   600 loss = 37.2269
DEBUG 	 step   700 loss = 36.599
DEBUG 	 step   800 loss = 38.0263
DEBUG 	 step   900 loss = 37.2955
DEBUG 	 step  1000 loss = 37.9216
DEBUG 	 step  1100 loss = 37.3359
DEBUG 	 step  1200 loss = 37.3713
DEBUG 	 step  1300 loss = 36.6709
DEBUG 	 step  1400 loss = 38.3146
DEBUG 	 step  1500 loss = 36.7897
DEBUG 	 step  1600 loss = 37.9787
DEBUG 	 step  1700 loss = 37.0694
DEBUG 	 step  1800 loss = 37.8516
DEBUG 	 step  1900 loss = 37.5252
DEBUG 	 step  2000 loss = 37.6231
DEBUG 	 step  2100 loss = 37.3801
DEBUG 	 step  2200 loss = 38.3831
DEBUG 	 step  2300 loss = 37.1824
DEBUG 	 step  2400 loss = 37.5972
DEBUG 	 step  2500 loss = 37.466
DEBUG 	 step  2600 loss = 37.8248
DEBUG 	 step  2700 loss = 36.4994
DEBUG

-0.056994025
randomReplacedDataset8.csv


INFO 	 Training with 959 minibatches per epoch
DEBUG 	 step     0 loss = 37.7497
DEBUG 	 step   100 loss = 37.6847
DEBUG 	 step   200 loss = 37.5399
DEBUG 	 step   300 loss = 39.2148
DEBUG 	 step   400 loss = 37.398
DEBUG 	 step   500 loss = 36.0838
DEBUG 	 step   600 loss = 37.647
DEBUG 	 step   700 loss = 37.0493
DEBUG 	 step   800 loss = 36.8636
DEBUG 	 step   900 loss = 37.187
DEBUG 	 step  1000 loss = 36.7762
DEBUG 	 step  1100 loss = 36.0937
DEBUG 	 step  1200 loss = 36.6706
DEBUG 	 step  1300 loss = 36.9844
DEBUG 	 step  1400 loss = 37.1844
DEBUG 	 step  1500 loss = 37.1308
DEBUG 	 step  1600 loss = 39.1861
DEBUG 	 step  1700 loss = 36.996
DEBUG 	 step  1800 loss = 38.7978
DEBUG 	 step  1900 loss = 37.0659
DEBUG 	 step  2000 loss = 36.9392
DEBUG 	 step  2100 loss = 36.9086
DEBUG 	 step  2200 loss = 38.4717
DEBUG 	 step  2300 loss = 37.4286
DEBUG 	 step  2400 loss = 38.759
DEBUG 	 step  2500 loss = 36.3101
DEBUG 	 step  2600 loss = 37.0405
DEBUG 	 step  2700 loss = 37.2975
DEBUG 